# 데이터 증강 끄기

In [1]:
# 📌 [기본 패키지 임포트]
import os
import shutil
import torch
import ultralytics
from ultralytics import YOLO
import time
import pandas as pd
import cv2
import matplotlib.pyplot as plt
import numpy as np
import json
from glob import glob
from collections import Counter
import threading

In [2]:
# 📌 [1] GPU 및 환경 체크
print("✅ CUDA 사용 가능:", torch.cuda.is_available())
print("✅ PyTorch 버전:", torch.__version__)
ultralytics.checks()

Ultralytics 8.3.156  Python-3.11.9 torch-2.7.1+cu128 CUDA:0 (NVIDIA GeForce RTX 4070 Laptop GPU, 8188MiB)
Setup complete  (24 CPUs, 31.7 GB RAM, 24.9/735.1 GB disk)


In [3]:
# 📌 [2] 실패한 이전 runs 삭제
def clean_incomplete_runs(base_dir='../code/runs/detect'):
    if not os.path.exists(base_dir):
        print(f"📂 '{base_dir}' 폴더가 없어 삭제 생략")
        return

    deleted = False
    for folder in os.listdir(base_dir):
        path = os.path.join(base_dir, folder)
        best_model = os.path.join(path, 'weights', 'best.pt')
        if not os.path.exists(best_model):
            print(f"🧹 삭제 대상: {path}")
            shutil.rmtree(path)
            deleted = True

    if not deleted:
        print("✅ 삭제할 미완성 run 없음. 모든 run이 best.pt 포함.")

clean_incomplete_runs()

✅ 삭제할 미완성 run 없음. 모든 run이 best.pt 포함.


In [4]:
# 📌 [3] 가장 최근 run 결과 요약 출력
def get_last_run_csv():
    runs = sorted(glob('runs/detect/*/results.csv'), key=os.path.getmtime, reverse=True)
    if runs:
        print(f"📄 마지막 훈련 결과 파일: {runs[0]}")
        df = pd.read_csv(runs[0])
        print("🔁 마지막 훈련 결과 (마지막 5줄):")
        print(df.tail())
    else:
        print("ℹ️ 이전 훈련 결과 없음.")

get_last_run_csv()

📄 마지막 훈련 결과 파일: runs/detect\yolov11n_custom_003\results.csv
🔁 마지막 훈련 결과 (마지막 5줄):
    epoch     time  train/box_loss  train/cls_loss  train/dfl_loss  \
25     26  2639.22         0.52503         0.44206         0.93175   
26     27  3299.47         0.52544         0.44100         0.93322   
27     28  3385.89         0.51527         0.43273         0.92348   
28     29  3472.51         0.51541         0.41596         0.92183   
29     30  3558.09         0.50541         0.41221         0.91931   

    metrics/precision(B)  metrics/recall(B)  metrics/mAP50(B)  \
25               0.94460            0.85920           0.93757   
26               0.94754            0.86024           0.93899   
27               0.96725            0.84840           0.94335   
28               0.94289            0.87677           0.94540   
29               0.95539            0.87270           0.94552   

    metrics/mAP50-95(B)  val/box_loss  val/cls_loss  val/dfl_loss    lr/pg0  \
25              0.80491    

In [5]:
# 📌 [4] 자동 넘버링 run 이름 생성
def get_next_run_name(base_dir='runs/detect', prefix='yolov11n_custom'):
    i = 1
    while os.path.exists(os.path.join(base_dir, f'{prefix}_{i:03d}')):
        i += 1
    return f'{prefix}_{i:03d}'

run_name = get_next_run_name()
print(f"🚀 새 훈련 run 이름: {run_name}")

🚀 새 훈련 run 이름: yolov11n_custom_004


In [6]:
# 📌 [5] 입력 데이터 정보 출력 및 클래스별 레이블 개수 확인 함수 추가
def get_dataset_info(img_dir, label_dir, class_names=None):
    img_files = glob(os.path.join(img_dir, '*.jpg'))
    label_files = glob(os.path.join(label_dir, '*.txt'))

    total_img_size = sum(os.path.getsize(f) for f in img_files)
    total_label_size = sum(os.path.getsize(f) for f in label_files)

    print(f"\n📁 이미지 경로: {img_dir}")
    print(f"🖼️ 이미지 개수: {len(img_files)}")
    print(f"💾 이미지 용량: {total_img_size / 1024 / 1024:.2f} MB")
    print(f"📄 라벨 경로: {label_dir}")
    print(f"📝 라벨 개수: {len(label_files)}")
    print(f"💾 라벨 용량: {total_label_size / 1024:.2f} KB")

    # 클래스별 라벨 개수 집계
    classes = []
    for f in label_files:
        with open(f, 'r') as file:
            lines = file.readlines()
            classes += [int(line.split()[0]) for line in lines if line.strip()]
    counter = Counter(classes)
    print(f"\n📊 클래스별 라벨 개수 ({label_dir}):")
    if class_names:
        for cls_idx, count in sorted(counter.items()):
            print(f"  {cls_idx} ({class_names[cls_idx]}): {count}")
    else:
        print(counter)

# 예시로 data.yaml에서 클래스명 읽어오기
import yaml
with open('data.yaml', 'r') as f:
    data_cfg = yaml.safe_load(f)
class_names = data_cfg.get('names', None)

get_dataset_info('../train/images', '../train/labels', class_names)
get_dataset_info('../valid/images', '../valid/labels', class_names)


📁 이미지 경로: ../train/images
🖼️ 이미지 개수: 3530
💾 이미지 용량: 67.55 MB
📄 라벨 경로: ../train/labels
📝 라벨 개수: 3530
💾 라벨 용량: 319.50 KB

📊 클래스별 라벨 개수 (../train/labels):
  0 (Green Light): 542
  1 (Red Light): 585
  2 (Speed Limit 10): 19
  3 (Speed Limit 100): 267
  4 (Speed Limit 110): 101
  5 (Speed Limit 120): 252
  6 (Speed Limit 20): 285
  7 (Speed Limit 30): 334
  8 (Speed Limit 40): 235
  9 (Speed Limit 50): 283
  10 (Speed Limit 60): 301
  11 (Speed Limit 70): 318
  12 (Speed Limit 80): 323
  13 (Speed Limit 90): 168
  14 (Stop): 285

📁 이미지 경로: ../valid/images
🖼️ 이미지 개수: 801
💾 이미지 용량: 15.38 MB
📄 라벨 경로: ../valid/labels
📝 라벨 개수: 801
💾 라벨 용량: 69.86 KB

📊 클래스별 라벨 개수 (../valid/labels):
  0 (Green Light): 122
  1 (Red Light): 108
  3 (Speed Limit 100): 52
  4 (Speed Limit 110): 17
  5 (Speed Limit 120): 60
  6 (Speed Limit 20): 56
  7 (Speed Limit 30): 74
  8 (Speed Limit 40): 55
  9 (Speed Limit 50): 71
  10 (Speed Limit 60): 76
  11 (Speed Limit 70): 78
  12 (Speed Limit 80): 56
  13 (Speed Limit 

In [ ]:
# 📌 캐시 파일 삭제 함수
# def safe_delete_cache(path: str):
#     if not os.path.exists(path):
#         print(f"🟢 캐시 파일 없음 (삭제 불필요): {path}")
#         return

#     file_size = os.path.getsize(path)
#     last_modified = os.path.getmtime(path)
#     now = time.time()

#     should_delete = (
#         file_size < 1024 or  # 1KB 이하
#         (now - last_modified > 3600)  # 1시간 이상 경과
#     )

#     if should_delete:
#         try:
#             os.remove(path)
#             print(f"✅ 캐시 삭제 완료: {path}")
#         except Exception as e:
#             print(f"❌ 캐시 삭제 실패: {path} → {e}")
#     else:
#         print(f"🔒 캐시 유지: {path} (조건 미충족)")

# safe_delete_cache(r"D:\Project\PJT_07\train\labels.cache")
# safe_delete_cache(r"D:\Project\PJT_07\valid\labels.cache")

In [7]:
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

# GPU 메모리 사용량 주기적으로 출력 (간격 60초로 조정)
def gpu_monitor():
    while True:
        if torch.cuda.is_available():
            mem_alloc = torch.cuda.memory_allocated() / 1024**2
            mem_reserved = torch.cuda.memory_reserved() / 1024**2
            print(f"[GPU 모니터] 메모리 할당: {mem_alloc:.1f}MB, 예약: {mem_reserved:.1f}MB")
        time.sleep(60)  # 60초 간격

monitor_thread = threading.Thread(target=gpu_monitor, daemon=True)
monitor_thread.start()

# 모델 로드
model = YOLO('yolo11n.pt')
# model = YOLO('runs/detect/yolov11n_custom_001/weights/best.pt')  # 기존 모델에서 이어서 학습 시

# 하이퍼파라미터 튜닝 포함한 train 함수 호출
start_time = time.time()

model.train(
    data='data.yaml',
    epochs=30,
    imgsz=640,
    batch=8,
    name=run_name,
    device='0',
    workers=0,
    verbose=True,  # 더 자세한 로그 출력
    augment=False,
)

train_duration = time.time() - start_time
print(f"✅ 훈련 소요 시간: {train_duration:.2f}초")

[GPU 모니터] 메모리 할당: 0.0MB, 예약: 0.0MB
Ultralytics 8.3.156  Python-3.11.9 torch-2.7.1+cu128 CUDA:0 (NVIDIA GeForce RTX 4070 Laptop GPU, 8188MiB)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=8, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=30, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=yolov11n_custom_004, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, p

train: Scanning D:\Project\PJT_07\train\labels.cache... 3530 images, 3 backgrounds, 0 corrupt: 100%|██████████| 3530


val: Fast image access  (ping: 0.10.0 ms, read: 103.466.5 MB/s, size: 20.7 KB)


val: Scanning D:\Project\PJT_07\valid\labels.cache... 801 images, 0 backgrounds, 0 corrupt: 100%|██████████| 801/801


Plotting labels to runs\detect\yolov11n_custom_004\labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.000526, momentum=0.9) with parameter groups 81 weight(decay=0.0), 88 weight(decay=0.0005), 87 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 0 dataloader workers
Logging results to runs\detect\yolov11n_custom_004
Starting training for 30 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...


30 epochs completed in 1.261 hours.
Optimizer stripped from runs\detect\yolov11n_custom_004\weights\last.pt, 5.5MB
Optimizer stripped from runs\detect\yolov11n_custom_004\weights\best.pt, 5.5MB

Validating runs\detect\yolov11n_custom_004\weights\best.pt...
Ultralytics 8.3.156  Python-3.11.9 torch-2.7.1+cu128 CUDA:0 (NVIDIA GeForce RTX 4070 Laptop GPU, 8188MiB)
YOLO11n summary (fused): 100 layers, 2,585,077 parameters, 0 gradients, 6.3 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95):  86%|████████▋ | 44/51 [00

[GPU 모니터] 메모리 할당: 294.8MB, 예약: 1538.0MB


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 51/51 [00


                   all        801        944      0.955      0.873      0.946      0.816
           Green Light         87        122      0.826      0.739      0.826      0.473
             Red Light         74        108      0.901      0.713      0.831      0.506
       Speed Limit 100         52         52      0.924      0.923      0.973       0.87
       Speed Limit 110         17         17          1      0.882      0.962       0.89
       Speed Limit 120         60         60      0.982        0.9      0.976      0.891
        Speed Limit 20         56         56      0.982      0.893      0.979      0.859
        Speed Limit 30         71         74      0.951      0.959      0.982      0.914
        Speed Limit 40         53         55      0.981      0.926      0.983      0.867
        Speed Limit 50         68         71      0.924      0.853      0.953      0.844
        Speed Limit 60         76         76      0.969      0.908      0.948      0.869
        Speed Limit 7

In [ ]:
# 📌 [7] 검증 수행
# start_val = time.time()

# model = YOLO(f'runs/detect/{run_name}/weights/best.pt')
# results = model.val()
# val_duration = time.time() - start_val
# print(f"✅ 검증 소요 시간: {val_duration:.2f}초")
# print(f"mAP50: {results.box.map50:.4f}, mAP50-95: {results.box.map:.4f}")

# # PR Curve 시각화 (val 결과 내 PR curve 있으면)
# if hasattr(results, 'plots') and 'pr_curve' in results.plots:
#     plt.figure(figsize=(8,6))
#     plt.imshow(results.plots['pr_curve'])
#     plt.axis('off')
#     plt.title("PR Curve")
#     plt.show()

In [8]:
print(run_name)

yolov11n_custom_004


In [9]:
# 📌 [8] 예측 실행 및 저장
test_img = '../test/images/00014_00004_00020_png.rf.e3b35f3ca153fde99af4db4a67d2a564.jpg'
pred = model.predict(
    source=test_img,
    conf=0.25,
    save=True,
    project='runs/predict/custom_test',
    name=run_name,
    exist_ok=True
)


image 1/1 D:\Project\PJT_07\code\..\test\images\00014_00004_00020_png.rf.e3b35f3ca153fde99af4db4a67d2a564.jpg: 640x640 1 Stop, 21.3ms
Speed: 4.1ms preprocess, 21.3ms inference, 2.8ms postprocess per image at shape (1, 3, 640, 640)
Results saved to runs\predict\custom_test\yolov11n_custom_004


In [10]:
# 📌 [9] 예측 결과 시각화 및 GT bbox와 비교 함수 추가
def draw_boxes_with_gt(image_path, pred_results, gt_label_path, class_names):
    img = cv2.imread(image_path)
    ih, iw = img.shape[:2]

    # GT 박스 (파란색)
    with open(gt_label_path, 'r') as f:
        gt_lines = f.readlines()

    for line in gt_lines:
        cls, x, y, w, h = map(float, line.strip().split())
        x1 = int((x - w/2) * iw)
        y1 = int((y - h/2) * ih)
        x2 = int((x + w/2) * iw)
        y2 = int((y + h/2) * ih)
        cv2.rectangle(img, (x1,y1), (x2,y2), (255,0,0), 2)
        cv2.putText(img, f"GT: {class_names[int(cls)]}", (x1, y1-5),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,0,0), 2)

    # 예측 박스 (초록색)
    for r in pred_results:
        boxes = r.boxes.xyxy.cpu().numpy().astype(int)
        scores = r.boxes.conf.cpu().numpy()
        classes = r.boxes.cls.cpu().numpy().astype(int)
        for (x1,y1,x2,y2), conf, cls in zip(boxes, scores, classes):
            label = f"{class_names[cls]} {conf:.2f}"
            cv2.rectangle(img, (x1,y1), (x2,y2), (0,255,0), 2)
            (tw, th), _ = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 0.5, 1)
            cv2.rectangle(img, (x1, y1 - th - 5), (x1 + tw, y1), (0,255,0), -1)
            cv2.putText(img, label, (x1, y1 - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1)

    plt.figure(figsize=(12,12))
    plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    plt.axis('off')
    plt.title("예측 박스 (초록) 와 GT 박스 (파랑) 비교")
    plt.show()

gt_label_for_test_img = '../test/labels/00014_00004_00020_png.rf.e3b35f3ca153fde99af4db4a67d2a564.txt'
draw_boxes_with_gt(test_img, pred, gt_label_for_test_img, class_names)

<Figure size 1200x1200 with 1 Axes>

In [11]:
# 📌 [10] ONNX 모델로 내보내기
export_path = model.export(format='onnx')
print(f"🧠 ONNX 모델 저장 경로: {export_path}")

Ultralytics 8.3.156  Python-3.11.9 torch-2.7.1+cu128 CPU (13th Gen Intel Core(TM) i7-13700HX)

PyTorch: starting from 'runs\detect\yolov11n_custom_004\weights\best.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 19, 8400) (5.2 MB)

ONNX: starting export with onnx 1.17.0 opset 19...
ONNX: slimming with onnxslim 0.1.57...
ONNX: export success  1.3s, saved as 'runs\detect\yolov11n_custom_004\weights\best.onnx' (10.1 MB)

Export complete (1.6s)
Results saved to D:\Project\PJT_07\code\runs\detect\yolov11n_custom_004\weights
Predict:         yolo predict task=detect model=runs\detect\yolov11n_custom_004\weights\best.onnx imgsz=640  
Validate:        yolo val task=detect model=runs\detect\yolov11n_custom_004\weights\best.onnx imgsz=640 data=data.yaml  
Visualize:       https://netron.app
🧠 ONNX 모델 저장 경로: runs\detect\yolov11n_custom_004\weights\best.onnx


In [12]:
# 📌 [11] ONNX 모델 기반 예측 시각화 (기존 코드 유지)
onnx_model = YOLO(os.path.join('runs/detect', run_name, 'weights', 'best.onnx'))

image = cv2.imread(test_img)
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
resized_image = cv2.resize(image, (640, 640))

results_onnx = onnx_model(resized_image, imgsz=640)

for result in results_onnx:
    boxes = result.boxes
    names = result.names
    for box in boxes:
        x1, y1, x2, y2 = map(int, box.xyxy[0])
        conf = box.conf[0]
        cls = int(box.cls[0])
        label = f"{names[cls]} {conf:.2f}"

        cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0), 2)
        (text_width, text_height), _ = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 0.5, 1)
        cv2.rectangle(image, (x1, y1 - text_height - 5), (x1 + text_width, y1), (0, 255, 0), -1)
        cv2.putText(image, label, (x1, y1 - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1)

plt.figure(figsize=(12, 12))
plt.imshow(image)
plt.axis('off')
plt.title("🔍 ONNX 예측 결과")
plt.show()

Loading runs\detect\yolov11n_custom_004\weights\best.onnx for ONNX Runtime inference...
Using ONNX Runtime CUDAExecutionProvider

0: 640x640 1 Stop, 32.5ms
Speed: 3.9ms preprocess, 32.5ms inference, 3.0ms postprocess per image at shape (1, 3, 640, 640)


<Figure size 1200x1200 with 1 Axes>

In [ ]:
# print(df.columns)

In [13]:
# 📌 [12] 훈련 결과 시각화 (mAP/Loss 등)
results_csv = f'runs/detect/{run_name}/results.csv'

if os.path.exists(results_csv):
    df = pd.read_csv(results_csv)
    print(f"📈 훈련 결과 요약 (Epoch별 mAP):\n{df[['metrics/mAP50(B)', 'metrics/mAP50-95(B)']].tail()}")

    # Box Loss 그래프
    df[['train/box_loss', 'val/box_loss']].plot(title='Box Loss', figsize=(6, 4))
    plt.savefig(f'runs/detect/{run_name}/box_loss.png')
    plt.show()

    # ✅ 수정: train/valid 전체 손실 시각화 (box, cls, dfl)
    df[['train/box_loss', 'train/cls_loss', 'train/dfl_loss',
        'val/box_loss', 'val/cls_loss', 'val/dfl_loss']].plot(figsize=(10,6))
    plt.title('Train & Valid Losses (Box, Class, DFL)')
    plt.savefig(f'runs/detect/{run_name}/losses.png')
    plt.show()

    # mAP, Precision, Recall 시각화
    df[['metrics/mAP50(B)', 'metrics/precision(B)', 'metrics/recall(B)']].plot(figsize=(10,6))
    plt.title('Metrics: mAP50, Precision, Recall')
    plt.savefig(f'runs/detect/{run_name}/metrics.png')
    plt.show()

    best_idx = df['metrics/mAP50-95(B)'].idxmax()
    print("🎯 최고 성능 Epoch:")
    print(df.loc[best_idx, ['epoch', 'metrics/mAP50(B)', 'metrics/mAP50-95(B)']])
else:
    print("📛 results.csv 없음 (훈련 실패?)")

📈 훈련 결과 요약 (Epoch별 mAP):
    metrics/mAP50(B)  metrics/mAP50-95(B)
25           0.93757              0.80491
26           0.93899              0.80909
27           0.94335              0.81435
28           0.94540              0.81500
29           0.94552              0.81616


<Figure size 600x400 with 1 Axes>

<Figure size 1000x600 with 1 Axes>

<Figure size 1000x600 with 1 Axes>

🎯 최고 성능 Epoch:
epoch                  30.00000
metrics/mAP50(B)        0.94552
metrics/mAP50-95(B)     0.81616
Name: 29, dtype: float64


In [14]:
# 📌 [13] 결과를 JSON으로 저장
def save_results_to_json(csv_path, json_path):
    if os.path.exists(csv_path):
        df = pd.read_csv(csv_path)
        result_data = df.to_dict(orient='records')
        with open(json_path, 'w') as f:
            json.dump(result_data, f, indent=4)
        print(f"📦 결과 JSON 저장 완료: {json_path}")

save_results_to_json(
    csv_path=f'runs/detect/{run_name}/results.csv',
    json_path=f'runs/detect/{run_name}/results_summary.json'
)

📦 결과 JSON 저장 완료: runs/detect/yolov11n_custom_004/results_summary.json
